The goal of this notebook is to find parameters for the MOTIS ODM Mixer that satisfy the requirements specified by the respective public transit provider.

In [ ]:
# Setup
import os
import subprocess
from violations import violations
from visualize import visualize

motis = "/home/mority/code/motis/cmake-build-release-gcc-14/motis"

In [ ]:
# Configure mixer
cfg = \
"""{
    "direct_taxi_penalty": 60.0,
    "max_distance": 60,
    "taxi_cost": [[0,10],[1,4]],
    "transfer_cost": [[0,10]]
}"""
with open("mixer.json","w") as f:
    f.write(cfg)
cfg_path = os.path.abspath("mixer.json")

In [ ]:
# Mix test input
for cs in os.listdir("in"):
    in_path = os.path.join("in", cs)
    if os.path.isdir(in_path):
        out_path = os.path.join("out", cs)
        os.makedirs(out_path, exist_ok=True)
        subprocess.run(motis + " mixer -c " + cfg_path + " -i " + os.path.join(in_path, "journeys.csv") + " -o " + out_path, shell=True)

In [ ]:
# Find required/forbidden violations and visualize connection sets
for cs in os.listdir("in"):
    in_path = os.path.join("in", cs)
    if os.path.isdir(in_path):
        out_path = os.path.join("out",cs)
        if not os.path.isdir(out_path):
            print("No results for {}".format(cs))
            continue
        if violations(cs):
            visualize(cs)

In [ ]:
# correct number of transfers for mixed journeys, i.e., use pt and taxi
# the change from/to taxi is not counted as transfer currently
# therefore we need to increase the number of transfers for each journey that uses both pt and taxi

import os
import pandas as pd

def to_time_types(journeys):
    journeys["departure"] = pd.to_datetime(journeys["departure"], utc=True)
    journeys["arrival"] = pd.to_datetime(journeys["arrival"], utc=True)
    journeys["first_mile_duration"] = journeys["first_mile_duration"].apply(lambda x: pd.Timedelta(x, unit="m"))
    journeys["last_mile_duration"] = journeys["last_mile_duration"].apply(lambda x: pd.Timedelta(x,unit="m"))


def fix_mixed_journeys(journeys):
    journeys["transfers"] = journeys["transfers"] + (
        (
            (journeys["first_mile_mode"] == "taxi")
            | (journeys["last_mile_mode"] == "taxi")
        )
        & (
            journeys["arrival"] - journeys["departure"]
            != journeys["first_mile_duration"]
        )
        & (
            journeys["arrival"] - journeys["departure"]
            != journeys["last_mile_duration"]
        )
    ).apply(lambda x : 1 if x else 0)

def reverse_to_time_types(journeys):
    journeys["departure"] = journeys["departure"].dt.tz_localize(None)
    journeys["arrival"] = journeys["arrival"].dt.tz_localize(None)
    journeys["first_mile_duration"] = journeys["first_mile_duration"].apply(lambda x: x.components.minutes)
    journeys["last_mile_duration"] = journeys["last_mile_duration"].apply(lambda x: x.components.minutes)

for cs in os.listdir("in"):
    in_path = os.path.join("in", cs)
    if os.path.isdir(in_path):
        journeys = pd.read_csv(
            os.path.join(in_path, "journeys.csv"), skipinitialspace=True
        )
        forbidden = pd.read_csv(
            os.path.join(in_path, "forbidden.csv"), skipinitialspace=True
        )
        required = pd.read_csv(
            os.path.join(in_path, "required.csv"), skipinitialspace=True
        )

        to_time_types(journeys)
        to_time_types(forbidden)
        to_time_types(required)

        fix_mixed_journeys(journeys)
        fix_mixed_journeys(forbidden)
        fix_mixed_journeys(required)

        reverse_to_time_types(journeys)
        reverse_to_time_types(forbidden)
        reverse_to_time_types(required)

        journeys.to_csv(os.path.join(in_path, "journeys.csv"))
        forbidden.to_csv(os.path.join(in_path, "forbidden.csv"))
        required.to_csv(os.path.join(in_path, "required.csv"))